In [1]:
import numpy as np

In [2]:
# the main algorithm
def channel_shuffle(x, groups):
    channels_per_group = int(len(x)/groups)
    x = x.reshape(groups, channels_per_group)
    x = x.T
    x = x.reshape(groups*channels_per_group)
    return x

In [3]:
# for visualization
def print_grouped(x, groups):
    channels_per_group = int(len(x)/groups)
    s = []
    for i, v in enumerate(x):
        if i % channels_per_group == 0:
            s += ['|']
        s += [str(v)]
    s += ['|']
    return ' '.join(s)

In [4]:
# this represents channels
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

## groups = 4

In [5]:
groups = 4

In [6]:
# before channel shuffle
print_grouped(x, groups)

'| 0 1 2 | 3 4 5 | 6 7 8 | 9 10 11 |'

In [7]:
# after channel shuffle
print_grouped(channel_shuffle(x, groups), groups)

'| 0 3 6 | 9 1 4 | 7 10 2 | 5 8 11 |'

## groups = 3

In [8]:
groups = 3

In [9]:
print_grouped(x, groups)

'| 0 1 2 3 | 4 5 6 7 | 8 9 10 11 |'

In [10]:
print_grouped(channel_shuffle(x, groups), groups)

'| 0 4 8 1 | 5 9 2 6 | 10 3 7 11 |'